## Dependencies

In [10]:
import os
import shutil
import cv2
import glob
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

2023-04-16 13:45:28.025705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 13:45:28.839944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/riemer/enter/lib/python3.9/site-packages/cv2/../../lib64::/usr/local/lib:/usr/local/lib
2023-04-16 13:45:28.839972: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-16 13:45:30.560684: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic libr

## Optical Flow Calculation

### Middle frame 

In [ ]:
# root = 'HMDB51/video_data'

# # Set the parameters for the optical flow calculation
# lk_params = dict(winSize=(15, 15),
#                  maxLevel=2,
#                  criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# for path, dirs, files in os.walk(root):
#     for file in files:
#         video_path = os.path.join(path, file)
#         cap = cv2.VideoCapture(video_path)
#         length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
#         # obtain the middle frame
#         frame_num = length//2
#         cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
#         _, frame = cap.read()
        
#         cv2.imwrite('middle.png', frame)
        
#         prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
#         cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num+1)
#         _, frame = cap.read()
        
#         cv2.imwrite('next.png', frame)
        
#         cap.release()
        
#         gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
              
#         # Calculate the optical flow between the current and previous frames
#         flow = cv2.calcOpticalFlowFarneback(prev_frame, gray_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

#         # Compute the magnitude and direction of the optical flow
#         magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

#         # Normalize the magnitude to a range between 0 and 255
#         magnitude_norm = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

#         # Convert the magnitude to a grayscale image
#         magnitude_image = np.uint8(magnitude_norm)

#         # Save the optical flow image
#         new_path = path.replace('video_data', 'optical_flow')
#         os.makedirs(new_path, exist_ok=True)
#         optical_flow_path = os.path.join(new_path, f'{file}_{i}.png')
#         cv2.imwrite(optical_flow_path, magnitude_image)

In [15]:
root = 'HMDB51/video_data'

for path, dirs, files in os.walk(root):
    for file in files:
        video_path = os.path.join(path, file)
        cap = cv2.VideoCapture(video_path)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # obtain the middle frame number
        frame_num = length//2
        
        # Read the middle frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        _, middle_frame = cap.read()
        prev_frame = cv2.cvtColor(middle_frame, cv2.COLOR_BGR2GRAY)
        
        # Read the subsequent frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num+1)
        _, next_frame = cap.read()        
        next_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

        cap.release()
        
        # Create a mask with max saturation
        mask = np.zeros_like(middle_frame)
        mask[..., 1] = 255
                      
        # Calculate the optical flow between the middle frame and the next frame
        flow = cv2.calcOpticalFlowFarneback(prev_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Compute the magnitude and direction of the optical flow
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Set image hue according to the optical flow direction
        mask[..., 0] = angle * 180 / np.pi / 2
        
        # Set image value according to the optical flow magnitude
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        
        # Convert to RGB
        rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)

        # Save the optical flow image
        new_path = path.replace('video_data', 'optical_flow')
        os.makedirs(new_path, exist_ok=True)
        optical_flow_path = os.path.join(new_path, f'{file[:-4]}.png')
        cv2.imwrite(optical_flow_path, rgb)

### All frames

In [ ]:
root = 'HMDB51/video_data'

# Set the parameters for the optical flow calculation
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

for path, dirs, files in os.walk(root):
    for file in files:
        video_path = os.path.join(path, file)
        cap = cv2.VideoCapture(video_path)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(length)
        prev_frame = None

        # Loop through each frame in the video and calculate the optical flow
        for i in range(length):
            # Read the next frame
            ret, frame = cap.read()
            
            if ret:
                # Convert the frame to grayscale
                gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                if prev_frame is not None:
                    # Calculate the optical flow between the current and previous frames
                    flow = cv2.calcOpticalFlowFarneback(prev_frame, gray_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

                    # Compute the magnitude and direction of the optical flow
                    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

                    # Normalize the magnitude to a range between 0 and 255
                    magnitude_norm = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

                    # Convert the magnitude to a grayscale image
                    magnitude_image = np.uint8(magnitude_norm)

                    # Save the optical flow image
                    new_path = path.replace('video_data', 'optical_flow')
                    os.makedirs(new_path, exist_ok=True)
                    optical_flow_path = os.path.join(new_path, f'{file}_{i}.png')
                    cv2.imwrite(optical_flow_path, magnitude_image)

                # Set the current frame as the previous frame for the next iteration
                prev_frame = gray_frame
                
            else:
                cap.release()

## Data Load

In [16]:
keep_hmdb51 = ["clap", "climb", "drink", "jump", "pour", "ride_bike", "ride_horse", 
        "run", "shoot_bow", "smoke", "throw", "wave"]

TRAIN_TAG, TEST_TAG = 1, 2
train_files, test_files = [], []
train_labels, test_labels = [], []
split_pattern_name = f"*test_split1.txt"
split_pattern_path = os.path.join('HMDB51/test_train_splits', split_pattern_name)
annotation_paths = glob.glob(split_pattern_path)
for filepath in annotation_paths:
    class_name = '_'.join(filepath.split('/')[-1].split('_')[:-2])
    if class_name not in keep_hmdb51:
        continue  # skipping the classes that we won't use.
    with open(filepath) as fid:
        lines = fid.readlines()
    for line in lines:
        video_filename, tag_string = line.split()
        tag = int(tag_string)
        if tag == TRAIN_TAG:
            train_files.append(video_filename)
            train_labels.append(class_name)
        elif tag == TEST_TAG:
            test_files.append(video_filename)
            test_labels.append(class_name)

In [17]:
x_train = []
x_test = []

for file in train_files:
    vid_name = file[:-4]
    for path, _, files in os.walk('HMDB51/optical_flow'):
        for file_name in files:
            if file_name[:-4] == vid_name:
                img = cv2.imread(os.path.join(path, file_name))
                img = cv2.resize(img, (112, 112))
                x_train.append(img)
                
for file in test_files:
    vid_name = file[:-4]
    for path, _, files in os.walk('HMDB51/optical_flow'):
        for file_name in files:
            if file_name[:-4] == vid_name:
                img = cv2.imread(os.path.join(path, file_name))
                img = cv2.resize(img, (112, 112))
                x_test.append(img)
                
x_train = np.asarray(x_train)/255.
x_test = np.asarray(x_test)/255.

In [18]:
# Dict to convert string labels to ints
str_to_int = {"clap":0, "climb":1, "drink":2, "jump":3, "pour":4, "ride_bike":5, "ride_horse":6, 
              "run":7, "shoot_bow":8, "smoke":9, "throw":10, "wave":11}

y_train = np.array([str_to_int[label] for label in train_labels])
y_test = np.array([str_to_int[label] for label in test_labels])

## Model init

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(112, 112, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=12, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32)

## Testing

In [ ]:
result = model.evaluate(x_test, y_test, batch_size=32)